## Summarization estrattiva con Nasari
File scelto, trump-wall
# Scelta implementativa importante:
Per evitare il paywall si è deciso di provare ad evitare l'uso di BableNet e di utilizzare come chiave di Nasari il lemma presente nella seconda colonna del file.
Si sono raggiunte comunque buoni risultati

In [1]:
#Import and constants
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
COMPRESSION_RATE=20

In [2]:
#Utilities

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def load_nasari(filename):
    nasari={}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            line=line.lower()
            line = line.strip().split(';')
            nasari[line[1]] = []
            for elem in line[2:]:
                elem = elem.strip().split('_')
                if len(elem) == 2:
                    nasari[line[1]].append((elem[0],float(elem[1])))
    return nasari
#Calcola la misura di valutazione Rouge tra il reference e il generated summary
def calculate_rouge(reference_summary, generated_summary):
    reference = set(word_tokenize(reference_summary))
    generated = set(word_tokenize(generated_summary))
    blue_score = len(reference.intersection(generated)) / len(reference)
    return blue_score

#Calcola la misura di valutazione Blue tra il reference e il generated summary
def calculate_blue(reference_summary, generated_summary):
    reference = set(word_tokenize(reference_summary))
    generated = set(word_tokenize(generated_summary))
    blue_score = len(reference.intersection(generated))/len(generated)
    return blue_score

#Conta le frasi di un testo
def count_sentences(text):
    sentence_tokens = sent_tokenize(text)
    num_sentences = len(sentence_tokens)
    return num_sentences

#Calcola l'overlap  tra due vettori pesato per il loro valore di peso
def calculate_weighted_overlap(vec1,vec2):
    overlap=0
    for elem_1 in vec1:
        for elem_2 in vec2:
            if(elem_1[0]==elem_2[0]):
                overlap+=elem_1[1]+elem_2[1]
    return overlap
#Lemmatizza un testo rimuovendo le stopWords
def lemmatize_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# Funzione di summarization

In [3]:
#Per capire la similarità di due parole calcoliamo il weighted overlap tra i loro embedding di Nasari
def get_word_similarity(word1, word2,nasari):
    try:
        vec1 = nasari[word1]
        vec2 = nasari[word2]
        return calculate_weighted_overlap(vec1, vec2)
    except:
        #print(word1,"-",word2)
        return 0.0

#Forniamo uno score ad una frase sulla base della similarità dei token della frase ai keyword tokens
def calculate_scores(text_tokens, keyword_tokens,nasari):
    scores = []
    #Per ogni token nella frase tokenizzata
    for token in text_tokens:
        max_similarity = 0
        #Per ogni parola nei keyword_tokens
        for keyword in keyword_tokens:
            #Calcolo la similarità
            similarity = get_word_similarity(token, keyword,nasari)
            #Eventualmente aggiorno il massimo
            if similarity > max_similarity:
                max_similarity = similarity
        #Aggiungo nella lista degli scores delle parole la similarità massima
        scores.append(max_similarity)
    #Ritorno la somma degli scores di tutte le parole
    return sum(scores)

def summarize_text(text, num_sentences,nasari):
    #Lemmatizzo il testo e rimuovo le stopwrods per ottenere le keyword del testo
    keyword_tokens = lemmatize_text(text)
    #Divido il testo in frasi
    sentence_tokens = sent_tokenize(text)
    sentence_scores = []
    #Per ogni frase
    for index,sentence in enumerate(sentence_tokens):
        #Lemmatizzo la frase
        tokens = lemmatize_text(sentence)
        #Calcolo il punteggio della frase
        sentence_score = calculate_scores(tokens, keyword_tokens,nasari)
        #Salvo il punteggio
        sentence_scores.append((index,sentence_score))
    #Ordino le frasi sulla base del loro score
    sorted_indices = sorted(sentence_scores, key=lambda x: x[1],reverse=True)
    #Estraggo i primi  num_sentences indici
    top_indices = sorted_indices[:num_sentences]
    #Estraggo le frasi relative agli indici
    summary_sentences = [sentence_tokens[i] for i in (index[0] for index in top_indices)]
    #Creo il summary
    summary = ' '.join(summary_sentences)  # Unisci le frasi per ottenere il riassunto
    return summary

# Main e valutazione

In [4]:
#Carico Nasari
nasari = load_nasari("Nasari/dd-small-nasari-15.txt")
#Carico il dataset
dataset_text = read_text_from_file("Nasari/Trump-wall.txt")

In [5]:
#Carico il gold-set
reference_text = read_text_from_file("Nasari/Trump-wall-reference-80.txt")
#Calcolo il numero di frasi che voglio estrarre
num_phrases = int(count_sentences(dataset_text) * (1 - COMPRESSION_RATE / 100))
#Calcolo il summary
generated_summary = summarize_text(dataset_text,num_phrases,nasari)

#Calcolo gli scores
rouge_scores = calculate_rouge(reference_text, generated_summary)
blue_score = calculate_blue(reference_text, generated_summary)

print("PRECISION:",blue_score)
print("RECALL:",rouge_scores)

PRECISION: 0.976148409893993
RECALL: 0.9364406779661016
